In [15]:
import numpy as np
import torch



# input : tensor(numpy array)[-1,64] --> output : flag(numpy array)[]
# (block(16word * 64) * n개) => [-1,64], 1개당 int16
def SR(x):  
     # sign reduction : 1개의 data(2B) 기준 상위 9bit 비교  
     x_numpy = x.view(np.int16)
     sr_result = x_numpy
     sr_flag = 0
     
     # ### 0000_0000_0111_1111 이하,  1111_1111_1000_0000 이상인지. 해당하면 1을 저장.
     is_sr_array         = ((x_numpy < 0x80) & (x_numpy >= 0)) | (x_numpy <= -128)
     
     # first data(word)에 대한 특수한 조건. (10bit 비교) ### 0000_0000_0011_1111 이하, 1111_1111_1100_0000 이상인지.
     first_element_flag  = ((x_numpy[:,0] < 0x40  & (x_numpy[:,0] >=0)) | (x_numpy[:,0] >= -64)).reshape(-1,1)

     # is_sr_array[-1,64] 일 때, 모든 2차원 (word 단위의 64개)가 sign reduction 가능한지. 
     row_flag            = (np.sum(is_sr_array, axis=1) == 64).reshape(-1,1)
     sr_flag             = row_flag & first_element_flag
     
     return sr_flag

def Comp(x_tensor : torch.Tensor):      # Tensor (128,3,224,224) -> Tensor (64,3,224,224)
    x = x_tensor.numpy().view(np.int16)
    x_numpy = x.reshape(-1,64) # 94080 16
    
#     print(x_numpy.dtype)
#     # Do SR
     
    sr_flag = SR(x_numpy)
    # Do ZRLE
     
#     zrle_flag = ZRLE(x_numpy)
    # Do BPC
     
    # bpc_flag = BPC(x_numpy)

    return sr_flag #| zrle_flag | bpc_flag.reshape(-1,1)

x = torch.rand(10,3,224,224).reshape(-1,64)
print(x)
Comp(x)


# arr = np.arange(0, 2*3*4)
# v= arr.reshape([2,3,4]) # row = 2, col = 3, depth = 4
# v.sum(axis = 2)
# array([[[ 0,  1,  2,  3],
     #    [ 4,  5,  6,  7],
     #    [ 8,  9, 10, 11]],
# 
     #   [[12, 13, 14, 15],
     #    [16, 17, 18, 19],
     #    [20, 21, 22, 23]]])
     
# axis = 0 -> row 합 [[[x,x,x,x],[y,y,y,y]],[[a,a,a,a],[b,b,b,b]]] -> [x],[a] 합 / [y],[b] 합
# array([[12, 14, 16, 18],
     #   [20, 22, 24, 26],
     #   [28, 30, 32, 34]])

# axis = 1 -> col 부분 합 [[[x,x,x,x],[y,y,y,y]],[[a,a,a,a],[b,b,b,b]]] -> [x],[y] 합 / [a],[b] 합
# array([[12, 15, 18, 21],
#        [48, 51, 54, 57]])

# axis = 2 -> 마지막 3 차원을 제거하여  [[[x,x,x,x]]] -> [x,x,x,x] 부분 합.
# array([[ 6, 22, 38],
     #   [54, 70, 86]])

tensor([[0.0367, 0.6365, 0.7962,  ..., 0.4862, 0.2256, 0.4023],
        [0.5696, 0.7853, 0.7588,  ..., 0.1774, 0.2232, 0.2633],
        [0.0855, 0.1620, 0.1615,  ..., 0.0238, 0.4141, 0.6277],
        ...,
        [0.8521, 0.7907, 0.9519,  ..., 0.7221, 0.5544, 0.6369],
        [0.1954, 0.9506, 0.0942,  ..., 0.7376, 0.3998, 0.1361],
        [0.9274, 0.2394, 0.2695,  ..., 0.5963, 0.0205, 0.4518]])
(47040, 64)


array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [ ]:
# input : tensor(numpy array)[-1,64] --> output : flag(numpy array)[]
# (block(16word * 64) * n개) => [-1,64], 1개당 int16

def ZRLE(x): 
     # 1개의 word(2B) 기준 단위로 zero, non_zero 파악 이후 encoding
     x_numpy = x.view(np.int16)
    
     # word 단위(int16)로 non_zero의 경우 flag = 1인 tensor
     # 즉, tensor는 Byte 단위이므로, int16 -> 1B로 표현해서
     # 4 word를 묶어 64bit 단위로 pattern을 파악하기 위해서는 4B씩 묶어야 하므로 
     # 기존의 1B씩 된 mask를 int32로 표현하여 4개씩 묶음.
     non_zero_element         = (x_numpy != 0)
     
     non_zero_element_pattern = non_zero_element.view(np.int32)
     
     non_zero_size      = (non_zero_element_pattern == 0x00000000) * 6
     
     non_zero_size     += (non_zero_element_pattern == 0x01000000) * 22
     non_zero_size     += (non_zero_element_pattern == 0x00010000) * 21
     non_zero_size     += (non_zero_element_pattern == 0x00000100) * 21
     non_zero_size     += (non_zero_element_pattern == 0x00000001) * 21
     
     non_zero_size     += (non_zero_element_pattern == 0x01010000) * 36
     non_zero_size     += (non_zero_element_pattern == 0x01000100) * 36
     non_zero_size     += (non_zero_element_pattern == 0x01000001) * 36
     non_zero_size     += (non_zero_element_pattern == 0x00010100) * 36
     non_zero_size     += (non_zero_element_pattern == 0x00010001) * 36
     non_zero_size     += (non_zero_element_pattern == 0x00000101) * 36
     
     non_zero_size     += (non_zero_element_pattern == 0x01010100) * 52
     non_zero_size     += (non_zero_element_pattern == 0x01010001) * 52
     non_zero_size     += (non_zero_element_pattern == 0x01000101) * 52
     non_zero_size     += (non_zero_element_pattern == 0x00010101) * 52
     
     non_zero_size     += (non_zero_element_pattern == 0x01010101) * 66

     zrle_flag            = (np.sum(non_zero_size,axis=1) < 510).reshape(-1,1)

     return zrle_flag

def Comp(x_tensor : torch.Tensor):      # Tensor (128,3,224,224) -> Tensor (64,3,224,224)
    x = x_tensor.numpy().view(np.int16)
    x_numpy = x.reshape(-1,64) # 94080 16
    
#     print(x_numpy.dtype)
#     # Do SR
     
    # sr_flag = SR(x_numpy)
    # Do ZRLE
     
    zrle_flag = ZRLE(x_numpy)
    # Do BPC
     
    # bpc_flag = BPC(x_numpy)

    return zrle_flag #| bpc_flag.reshape(-1,1)

x = torch.rand(1,64).reshape(-1,64)
print(x.dtype)
Comp(x)

In [193]:
def BPC(input):
     input_np = input.view(np.int16)
     row = input_np.shape[0]
     
     # Data Block
     input_np_base = input_np[:,0].reshape(-1,1)
     base = input_np_base.copy()
     base_uint8 = base.view(np.uint8)
     base_unpackbits = np.unpackbits(base_uint8, bitorder = 'little')
     BASE = base_unpackbits.reshape((row,16))[:,::-1]
     
     # delta
     input_np_delta = input_np[:,1:].reshape(-1,63) - input_np_base
     
     
     # Delta-BP
     delta_uint8 = input_np_delta.view(np.uint8) # 현재 little endian
     delta_unpackbits = np.unpackbits(delta_uint8, bitorder ='little')
     delta_unpackbits_2d = delta_unpackbits.reshape((row,63,16))[:,:,::-1]
     Delta_BP = np.swapaxes(delta_unpackbits_2d, 1, 2) # (row,16,63)
     Delta_BP_CNT = np.sum(Delta_BP, axis=2) # (row,16)
     
     
     # DBP_XOR
     DBP_XOR = Delta_BP.copy()
     DBP_XOR[:,1:,:] ^= DBP_XOR[:,:15,:] # (row,16,63)
     DBP_XOR_CNT = np.sum(DBP_XOR, axis=2) # (row,16)
     DBP_XOR_SUM = np.zeros((DBP_XOR.shape[0],DBP_XOR.shape[1],DBP_XOR.shape[2]+1), dtype = np.uint8)
     DBP_XOR_SUM[:,:,0] = BASE
     DBP_XOR_SUM[:,:,1:] = DBP_XOR
     DBP_XOR_packbits =  np.packbits(DBP_XOR_SUM.reshape(-1,))
     DBP_XOR_SYMBOL = DBP_XOR_packbits.view(np.uint64).reshape(row,-1,1)
     
     
     
     # Encoding
          
     ## Case 1 : All-0's 
     mask_all_zero  = (DBP_XOR_CNT == 0)
     mask_all_zero_left  = np.zeros(mask_all_zero.shape, dtype=np.uint8)
     mask_all_zero_right = np.zeros(mask_all_zero.shape, dtype=np.uint8)     
     mask_all_zero_left[:,:15] = (DBP_XOR_CNT[:,1:] == 0)
     mask_all_zero_left[:,1:] = (DBP_XOR_CNT[:,:15] == 0)
     case_1 = (DBP_XOR_CNT == 0) & ((mask_all_zero & mask_all_zero_left) == 0) & ((mask_all_zero & mask_all_zero_right) == 0)
     ### All-0's run length 2~16
     case_0 = (DBP_XOR_CNT == 0) & ~case_1
     
     ## Case 2 : All-1's 
     case_2 = (DBP_XOR_CNT == 63)
     
     ## Case 3 : DBP_XOR != 0 & Delta_BP == 0
     dbx_dbp_flag = (DBP_XOR_CNT != 0) & (Delta_BP_CNT ==0)
     case_3 = ~case_2 & (dbx_dbp_flag == 1)
     
     ## Case 4 : Consecutive two 1's 
     mask_two_consec = DBP_XOR_SYMBOL & (DBP_XOR_SYMBOL-1)
     mask_two_consec_result = (DBP_XOR_CNT == 2).reshape(row,16,1) & (DBP_XOR_SYMBOL == (mask_two_consec + (mask_two_consec >> 1))) 
     mask_two_consec_flag = mask_two_consec_result.reshape(-1,16)
     case_4 = ~case_2 & ~case_3 & (mask_two_consec_flag == 1)

     ## Case 5 : Single 1
     case_5 = ~case_2 & ~case_3 & ~case_4 & (DBP_XOR_CNT == 1)
     
     ## Case 6 : Uncompressed
     case_6 = (~case_0) & (~case_1) & (~case_2) & (~case_3) & (~case_4) & (~case_5)

     result_flag = np.zeros((DBP_XOR.shape),dtype=np.int16)
     # result_flag = (case_0)*0 + (case_1)*3 + (case_2)*5 + (case_3)*5 + (case_4)*11 + (case_5)*11+ (case_6)*64   
     result_flag = (case_0)*0 + (case_1)*3 + (case_2)*5 + (case_3)*6 + (case_4)*11 + (case_5)*12+ (case_6)*64   

     mask_all_zero_2 = np.zeros((DBP_XOR.shape[0],DBP_XOR.shape[1]+2), dtype=np.uint8)
     mask_all_zero_2_left = np.zeros((DBP_XOR.shape[0],DBP_XOR.shape[1]+2), dtype=np.uint8)
     mask_all_zero_2[:,1:17] = case_0
     mask_all_zero_2_left[:,:17] = mask_all_zero_2[:,1:]
     mask_all_zero_2_sum = np.sum((mask_all_zero_2 ^ mask_all_zero_2_left), axis=1)//2
     
     result_sum = np.sum(result_flag, axis=1) + mask_all_zero_2_sum*int(6)
     result = (result_sum <= 494)     
     return result

x = torch.rand(64,64).numpy().view(np.int16)
x_numpy = x.reshape(-1,64)
BPC(x_numpy)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]